# This notebook will demonstrate the spectrum of Snowflake privacy and data sharing capabilities
## Provider Account

### Demo Setup

In [ ]:
use role ACCOUNTADMIN; 

create or replace database PRIVACYEVOLUTION;
create or replace schema GLOBALSALES;

create or replace role FIN_ANALYST_RL; 

grant usage on database PRIVACYEVOLUTION to role FIN_ANALYST_RL;
grant usage on schema PRIVACYEVOLUTION.GLOBALSALES to role FIN_ANALYST_RL;

### Load test data from Snowsight UI

After downloading the *.csv from GitHub, execute the following steps: 

1. Under the **Data** tab in the left menu, Click **Databases**
2. Click database **PRIVACYEVOLUTION** and schema **GLOBALSALES**
3. In the upper right, click the **Create** dropdown and select **Table** then **From File**
4. Drag your *.CSV (customer_provider.csv) to the center of the screen and drop. You can alternatively click **Browse** and locate your file
5. On the **Load Data into Table** screen, go to the bottom. Keep "Create New Table" selected. Enter "customer" under the **Name** field and click **Next**
6. This will take you to a panel that lists all columnms. Leave all default values as-is and click **Load**. 

You should see a message that 500 rows were successfully loaded. 

### Run this cell to reset any existing privacy policies used in previous executions of this demo

In [ ]:
use role ACCOUNTADMIN;

alter table customer 
modify 
  column c_fname unset masking policy, column c_zip unset masking policy ,
  column c_phone unset masking policy ,
  column c_email unset masking policy ,
  column c_gender unset masking policy ,
  column c_age unset masking policy ;

alter table customer 
modify 
    column c_email unset tag PII,
    column c_phone unset tag PII;

ALTER TABLE customer modify column c_email unset projection policy ;
ALTER TABLE customer modify column c_phone unset projection policy ;
ALTER TABLE customer modify column c_fname unset projection policy ;
ALTER TABLE customer modify column c_lname unset projection policy ;

alter table customer unset aggregation policy ;

alter table customer DROP ROW ACCESS POLICY Customer_OH_Policy;

alter table customer drop privacy policy cust_dp_policy ;

alter tag pii unset masking policy simple_mask_string;

# Demo Starts HERE

## This notebook will demonstrate the spectrum of Snowflake capabilities to enforce privacy policies in a collaborative, data sharing environment. The following features will be highlighted: 
- Secure Sharing
- Secure Views
- Dynamic/Conditional Data Masking
- Object Tagging
- Tag Propagation
- Row Access Policies
- Projection Policies
- Aggregation Policies
- Differential Privacy
- Data Clean Rooms (separate demo)

### Lets set our context and session variables

In [ ]:
use role ACCOUNTADMIN; 
use schema PRIVACYEVOLUTION.GLOBALSALES;

-- Modify these variables with your respective 
-- Snowflake account locators
set PROVIDERACCOUNT= 'XXX12345';
set CONSUMERACCOUNT= 'YYY98765';

### Lets look at some of my data...

In [ ]:
select * from customer limit 10;

### Now lets share this customer table to a consumer account

In [ ]:
grant select on customer to role FIN_ANALYST_RL;

create or replace share customer_share;
grant usage on database PRIVACYEVOLUTION to share customer_share;
grant usage on schema GLOBALSALES to share customer_share;
grant select on table customer to share customer_share;

alter share customer_share add accounts = identifier($CONSUMERACCOUNT);


### Switch to the consumer account to see how easy it is to access. 

### I just shared the entire customer table which is now completely visible to the consumer.  Maybe this isn't a great idea if it contains sensitive data.  Here's how we can use a VIEW to begin to restrict what the consumer can see. 

In [ ]:
create or replace SECURE view customerV as 
select encrypt(c_email,'abc') as c_email_crypt, c_state, c_zip, c_gender, c_age, c_mktsegment
from customer; 

grant select on customerV to share customer_share;

select * from customerV;

### Or I can filter out rows like this...

In [ ]:
create or replace SECURE view customerV as 
select encrypt(c_email,'abc') as c_email_crypt, c_state, c_zip, c_gender, c_age, c_mktsegment
from customer
where c_state = 'OH'; 

grant select on customerV to share customer_share;

select * from customerV limit 10;

### Note that the access for the consumer is immediately revokable

In [ ]:
revoke select on customerV from share customer_share;

In [ ]:
grant select on customerV to share customer_share;

### That works great!  But in a large, complex environment with 1000s of tables and tens of 1000s of columns, thats ALOT of views to manage.  We need a better option

### Introducing DYNAMIC DATA MASKING

In [ ]:
-- Define Masking Policies
create or replace masking policy simple_mask_string as
  (val string) returns string ->
  case
    when current_role() in ('ACCOUNTADMIN') then val
      else '*** masked *****'
    end;
    
create or replace masking policy simple_mask_int as
  (val integer) returns integer ->
  case
    when current_role() in ('ACCOUNTADMIN') then val
      else -999
    end;
    
-- Apply Masking Policies
alter table customer modify
  column c_fname set masking policy simple_mask_string,
  column c_zip set masking policy simple_mask_int,
  column c_phone set masking policy simple_mask_string,
  column c_email set masking policy simple_mask_string,
  column c_gender set masking policy simple_mask_string,
  column c_age set masking policy simple_mask_int;

### Policy has no effect when using role ACCOUNTADMIN

In [ ]:
use role ACCOUNTADMIN;
select * from customer limit 10;

### But if I switch to another role, the policy is enforced

In [ ]:
use role FIN_ANALYST_RL;
select * from customer limit 10;

### Check the consumer account to see if the masking policies are in effect

In [ ]:
-- Unset those policies
use role ACCOUNTADMIN;
alter table customer modify
  column c_fname unset masking policy ,
  column c_zip unset masking policy ,
  column c_phone unset masking policy ,
  column c_email unset masking policy ,
  column c_gender unset masking policy ,
  column c_age unset masking policy ;

### Maybe we want the mask to be dependent on some other column in our table that indicates whether the individual has "opted-in" to making their contact info public. In this case we can use CONDITIONAL Masking. This is a simple variation to Dynamic Data Masking

In [ ]:
create or replace masking policy email_visibility_policy as
(email varchar, visibility string) returns varchar ->
  case
    when current_role() in ('ACCOUNTADMIN') then email
    when visibility = 'Public' then email
    else '***CONDITIONAL MASK***'
  end;
  
alter table customer 
modify column c_email unset masking policy;

alter table if exists customer modify column c_email 
    set masking policy email_visibility_policy using (c_email, c_email_visibility);

### Scroll down and see that the EMAIL is masked when the C_EMAIL_VISIBILITY column is set to "Private"

In [ ]:
use role FIN_ANALYST_RL;
select C_FNAME, C_LNAME, C_EMAIL, C_EMAIL_VISIBILITY from customer;

### If we have MANY, MANY PII columns across MANY tables, thats alot of policy assignments to maintain.  We can make this easier by using OBJECT TAGS.  First, we will "unset" the masking policy we just applied to C_EMAIL

In [ ]:
use role ACCOUNTADMIN;

alter table if exists customer 
modify column c_email unset masking policy ;

### Then, we create an object tag and associate it with the same masking policy we created earlier

In [ ]:
create or replace tag PII propagate=on_dependency_and_lineage;

alter tag pii set masking policy simple_mask_string;

### Next, we apply the tag to one or more columns

In [ ]:
alter table customer 
modify 
    column c_email set tag PII = 'email address',
    column c_phone set tag PII = 'phone';


### Here we see that the masking policy followed the object tag!

In [ ]:
use role fin_analyst_rl;
select c_custkey, c_email, c_phone
from customer limit 10;

### Now show how the tags can get automatically propagated. (This feature is in Public Preview as of 12/2/2024)

### We create another table with data extracted from customer

In [ ]:
use role ACCOUNTADMIN;
create or replace table customer_extract_OH as
select * from customer
where c_state = 'OH';

grant select on customer_extract_OH to role FIN_ANALYST_RL;

### This shows how the tags, and the policies associated with the tag, were automatically propagated to a downstream table

In [ ]:
use role fin_analyst_rl;
select c_custkey, c_email, c_phone
from customer_extract_OH limit 10;

### What if we have privacy requirements to restrict access to specific rows or groups of rows?  We have ROW ACCESS POLICIES.  This has the same effect as the VIEW example we used earlier.  But the Row Access Policy makes it much easier to implement and manage. This policy will return only rows for Ohio. 

In [ ]:
use role ACCOUNTADMIN; 

-- Create a Row Access Policy 
-- This will only show rows for STATE='OH'.  ACCOUNTADMIN will see all rows
create or replace ROW ACCESS POLICY Customer_OH_Policy AS 
 (C_STATE VARCHAR) returns BOOLEAN -> 'ACCOUNTADMIN' = current_role() or C_STATE = 'OH';

alter table customer ADD ROW ACCESS POLICY Customer_OH_Policy on (C_STATE); 

In [ ]:
use role FIN_ANALYST_RL;
select * from customer; 

In [ ]:
-- Unset the Row Access Policy
use role ACCOUNTADMIN;
alter table customer DROP ROW ACCESS POLICY Customer_OH_Policy;  

### Here we can use a Projection Policy to control what columns can be included in the SELECT or *projection* of the query

In [ ]:
use role ACCOUNTADMIN; 

create or replace projection policy proj_policy
as () returns projection_constraint ->
case
  when CURRENT_ROLE() = 'ACCOUNTADMIN'
    then projection_constraint(allow => true)
  else projection_constraint(allow => false)
end;

ALTER TABLE customer modify column c_email set projection policy proj_policy;
ALTER TABLE customer modify column c_phone set projection policy proj_policy;
ALTER TABLE customer modify column c_fname set projection policy proj_policy;
ALTER TABLE customer modify column c_lname set projection policy proj_policy;

### This works normally as it is executed as role=ACCOUNTADMIN

In [ ]:
use role ACCOUNTADMIN;
select * from customer;

### But this one fail because role is not ACCOUNTADMIN and we are trying to project columns disallowed by the projection policy 

In [ ]:
use role FIN_ANALYST_RL;
select * from customer; -- This will fail

In [ ]:
select c_custkey, c_state, c_zip, c_age, c_gender 
from customer; -- This works!!!

### But I can do things like this... 

In [ ]:
select c_custkey, c_state, c_zip, c_age, c_gender 
from customer
where c_email = 'amaclead@gmail.com';

### The Projection Policy alone may not be sufficient to withstand aggressive attacks. So we can layer Aggregation Policies on top of this to thwart this


In [ ]:
use role ACCOUNTADMIN; 

create or replace aggregation policy cust_agg_policy
  as () returns aggregation_constraint -> 
  case 
    when current_role() = 'ACCOUNTADMIN'
      then no_aggregation_constraint()
    else 
      aggregation_constraint(min_group_size => 5)
  end;

alter table customer 
set aggregation policy cust_agg_policy;

### The previous attack now fails because I didn't include an aggregate in my query. 

In [ ]:
use role FIN_ANALYST_RL; 
select c_custkey, c_state, c_zip, c_age, c_gender 
from customer
where c_email = 'amaclead@gmail.com';

### This works as ACCOUNTADMIN. Not that it returns groups that have only 1-5 rows. 

In [ ]:
use role ACCOUNTADMIN;
select c_state, count(*), sum(c_acctbal) 
from customer group by 1 order by 2;

### When not ACCOUNTADMIN, this query now succeeds but will not return groups with < 5 rows as per the Aggregation policy

In [ ]:
use role FIN_ANALYST_RL;
select c_state, count(*), sum(c_acctbal) 
from customer group by 1 order by 2;

### Now check the consumer account to see that these policies are enforced there

### For even more protection against persistent and repetitive attacks at sensitive data, we have Differential Privacy. DP injects noise into your aggregated results subject to a privacy budget
### The goal is to maximize analytic value while eliminating the possibility of identifying or re-identifying sensitive data. 
### DP is mutually exclusive with Projection/Aggregation/DDM
### Row Access Policies take precedence if used together with DP

In [ ]:
-- Unset projection and aggregation policies 
use role ACCOUNTADMIN;

ALTER TABLE customer unset aggregation policy ;
ALTER TABLE customer modify column c_email unset projection policy;
ALTER TABLE customer modify column c_phone unset projection policy;
ALTER TABLE customer modify column c_fname unset projection policy;
ALTER TABLE customer modify column c_lname unset projection policy;

alter table customer modify column c_email unset masking policy;

### Create our Differential Privacy Policy...

In [ ]:
use role ACCOUNTADMIN;

create or replace privacy policy cust_dp_policy as () returns privacy_budget ->
  case
    when current_role() = 'ACCOUNTADMIN' THEN no_privacy_policy()
    else privacy_budget(budget_name => 'default')  
  end;

alter table customer add privacy policy cust_dp_policy ENTITY KEY (c_custkey);

### A non-ACCOUNTADMIN role will not be able to run a non-aggregated query against our protected table

In [ ]:
use role FIN_ANALYST_RL;
select * from customer; 

### But we can run this... 
### the DP_INTERVAL functions provide visibility into the "noise" range that was injected into the aggregate results

In [ ]:
SELECT COUNT(DISTINCT c_custkey) AS c,
       DP_INTERVAL_LOW(c) AS low,
       DP_INTERVAL_HIGH(c) AS high,
FROM customer;

In [ ]:
SELECT COUNT(DISTINCT c_custkey) AS c,
       DP_INTERVAL_LOW(c) AS low,
       DP_INTERVAL_HIGH(c) AS high,
FROM customer
WHERE c_gender='F'
  AND c_age between 30 and 50;

### Check what this looks like on the consumer account

### Demo tear down

In [ ]:
use role ACCOUNTADMIN;
alter table customer drop privacy policy cust_dp_policy ;

In [ ]:
-- ALWAYS BACK UP THE NOTEBOOK BEFORE RUNNING THIS. JUST IN CASE

Xdrop share customer_share; 
Xdrop database PRIVACYEVOLUTION;